In [1]:
import pandas as pd
from pandas import DataFrame
from pathlib import Path

data_dir = Path('../../mimic4-pg/data/')
adm_sub_ids = ['subject_id', 'hadm_id']

# Data Loading

In [2]:
mimic4_hosp_dir = data_dir / 'mimic-iv-2.2/hosp/'
admission_times = ['admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime']

required_files = ['patients', 'd_icd_diagnoses', 'd_icd_procedures']
hosp_tables: dict[str, DataFrame] = {f: pd.read_csv(mimic4_hosp_dir / (f + '.csv.gz')) for f in required_files}
patients = hosp_tables['patients']
admissions = pd.read_csv(mimic4_hosp_dir / 'admissions.csv.gz', parse_dates=admission_times)
diagnoses = pd.read_csv(mimic4_hosp_dir / 'diagnoses_icd.csv.gz', dtype={'icd_code': str})
procedures = pd.read_csv(mimic4_hosp_dir / 'procedures_icd.csv.gz', dtype={'icd_code': str})
d_icd_diagnoses = hosp_tables['d_icd_diagnoses']
d_icd_procedures = hosp_tables['d_icd_procedures']

In [3]:
icu_stays = pd.read_csv(data_dir / 'mimic-iv-2.2/icu/icustays.csv.gz', parse_dates=['intime', 'outtime'])

icu_stays = pd.merge(icu_stays, patients[['subject_id', 'anchor_year_group']], on='subject_id')
icu_patients = pd.merge(icu_stays['subject_id'], patients, on='subject_id').drop_duplicates('subject_id')

In [4]:
carevue_dir = data_dir / 'mimiciii-carevue'

carevue_patients = pd.read_csv(carevue_dir / 'PATIENTS.csv.gz')
carevue_adm = pd.read_csv(carevue_dir / 'ADMISSIONS.csv.gz')

In [5]:
mimiciii_dir = data_dir / 'mimiciii'

iii_adm = pd.read_csv(mimiciii_dir / 'ADMISSIONS.csv', parse_dates=list(map(str.upper, admission_times)))
iii_patients = pd.read_csv(mimiciii_dir / 'PATIENTS.csv.gz')
iii_diagnoses = pd.read_csv(mimiciii_dir / 'DIAGNOSES_ICD.csv', dtype={'ICD9_CODE': str}).rename({'ICD9_CODE': 'icd_code'}, axis=1)
iii_procedures = pd.read_csv(mimiciii_dir / 'PROCEDURES_ICD.csv', dtype={'ICD9_CODE': str}).rename({'ICD9_CODE': 'icd_code'}, axis=1)

for df in [iii_adm, iii_patients, iii_diagnoses, iii_procedures]:
    df.drop('ROW_ID', axis=1, inplace=True)
    df.rename(str.lower, axis=1, inplace=True)

# Admissions

In [6]:
def combine_icd_codes(df: DataFrame, type: str):
    combined = (df
                .groupby(['hadm_id', 'subject_id', 'icd_version'])
                .agg(set)
                .reset_index('icd_version')
                .rename({'icd_version': type + '_version', 'icd_code': type + '_codes'}, axis=1)
                )
    return combined

if 'pro_codes' not in admissions.columns:
    admissions = pd.merge(admissions, combine_icd_codes(diagnoses, 'dia'), how='left', on=adm_sub_ids)
    admissions = pd.merge(admissions, combine_icd_codes(procedures, 'pro'), how='left', on=adm_sub_ids, suffixes=('dia', 'pro'))

    admissions.loc[admissions.dia_codes.isna(), 'dia_codes'] = admissions.dia_codes[admissions.dia_codes.isna()].apply(lambda x: set())
    admissions.loc[admissions.pro_codes.isna(), 'pro_codes'] = admissions.pro_codes[admissions.pro_codes.isna()].apply(lambda x: set())

In [7]:
def combine_icd_codes_iii(df: DataFrame, type: str):
    combined = (df
                .dropna()
                .groupby(['hadm_id', 'subject_id'])
                .agg(set)
                .rename({'icd_code': type + '_codes'}, axis=1)
                )
    return combined

if 'pro_codes' not in iii_adm.columns:
    iii_adm = pd.merge(iii_adm, combine_icd_codes_iii(iii_diagnoses, 'dia'), how='left', on=adm_sub_ids)
    iii_adm = pd.merge(iii_adm, combine_icd_codes_iii(iii_procedures, 'pro'), how='left', on=adm_sub_ids)

    iii_adm.loc[iii_adm.dia_codes.isna(), 'dia_codes'] = iii_adm.dia_codes[iii_adm.dia_codes.isna()].apply(lambda x: set())
    iii_adm.loc[iii_adm.pro_codes.isna(), 'pro_codes'] = iii_adm.pro_codes[iii_adm.pro_codes.isna()].apply(lambda x: set())

In [8]:
times = 'admittime', 'dischtime', 'deathtime', 'edregtime', 'edouttime'
moments = [t.replace('time', 'moment') for t in times]
intervals = [t.replace('time', 'interval') for t in times if t != 'admittime']

def times_to_moment(df: DataFrame, *times: str):
    for t in times:
        df[t.replace('time', 'moment')] = df[t].dt.time
        if t != 'admittime':
            df[t.replace('time', 'interval')] = (df[t] - df['admittime']).dt.total_seconds()

times_to_moment(admissions, *times)
times_to_moment(iii_adm, *times)

# Patients

In [9]:
def counts_with_percentage(df: DataFrame):
    counts = df.value_counts()
    return pd.concat((counts.rename('total'), round((counts / df.size) * 100, 2).rename('relative')), axis=1)


def unique_patient_adms(df: DataFrame, cols=adm_sub_ids):
    return df[cols].nunique().rename('total')

In [10]:
patients['year_diff'] = patients['anchor_year'] - patients['anchor_year_group'].str[:4].astype('int')
counts_with_percentage(patients.anchor_year_group)

,total,relative
anchor_year_group,,
2008 - 2010,96695,32.26
2011 - 2013,72458,24.18
2014 - 2016,68131,22.73
2017 - 2019,62426,20.83
2020 - 2022,2,0.00


In [11]:
counts_with_percentage(icu_patients.anchor_year_group)

,total,relative
anchor_year_group,,
2008 - 2010,16155,31.73
2011 - 2013,12217,23.99
2014 - 2016,11940,23.45
2017 - 2019,10608,20.83


# ICU Stays

There are icu stays that are broken up by ward stays:

In [12]:
pd.concat((
    unique_patient_adms(icu_stays),
    icu_stays[icu_stays['hadm_id'].duplicated(keep=False)][adm_sub_ids].nunique().rename(
        'broken_up_icu_stays')
), axis=1)

,total,broken_up_icu_stays
subject_id,50920,5476
hadm_id,66239,5820


In [13]:
# time between these broken up stays
abs(icu_stays[icu_stays['hadm_id'].duplicated(keep=False)].groupby('hadm_id')['intime'].agg(['max', 'min']).diff(
    axis=1)['min']).describe().rename('duplicated_icu_stay_time_differences')

count                          5820
mean      9 days 12:34:05.728865979
std      11 days 21:30:28.364107979
min                 0 days 00:02:49
25%          2 days 18:22:43.500000
50%                 5 days 20:56:22
75%         11 days 19:02:22.250000
max               200 days 15:17:54
Name: duplicated_icu_stay_time_differences, dtype: object

In [14]:
counts_with_percentage(icu_stays.anchor_year_group)

,total,relative
anchor_year_group,,
2008 - 2010,26710,36.50
2011 - 2013,17215,23.52
2014 - 2016,15989,21.85
2017 - 2019,13267,18.13


# ICU Admissions

In [15]:
if 'year_diff' not in admissions.columns:
    admissions = pd.merge(admissions, patients[['subject_id', 'anchor_year_group', 'year_diff']], on='subject_id')
admissions['earliest_possible_dischyear'] = admissions['dischtime'].dt.year - admissions.year_diff

adm_icu = pd.merge(admissions, icu_stays[['subject_id', 'hadm_id']], on=['subject_id', 'hadm_id'])

unique_patient_adms(adm_icu)

subject_id    50920
hadm_id       66239
Name: total, dtype: int64

In [16]:
contaminated_ranges = ['2008 - 2010', '2011 - 2013']
counts_with_percentage(adm_icu.anchor_year_group).loc[contaminated_ranges]

,total,relative
anchor_year_group,,
2008 - 2010,26710,36.50
2011 - 2013,17215,23.52


# ICU Separation

In [17]:
adm_icu

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,dischinterval,deathmoment,deathinterval,edregmoment,edreginterval,edoutmoment,edoutinterval,anchor_year_group,year_diff,earliest_possible_dischyear
0,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,...,192000.0,NaN,NaN,05:54:00,-24060.0,14:00:00,5100.0,2014 - 2016,166,2014
1,10000980,26913865,2189-06-27 07:38:00,2189-07-03 03:00:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,501720.0,NaN,NaN,06:25:00,-4380.0,08:42:00,3840.0,2008 - 2010,178,2011
2,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaT,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,587040.0,NaN,NaN,17:38:00,-19080.0,01:24:00,8880.0,2011 - 2013,146,2011
3,10001217,27703517,2157-12-18 16:58:00,2157-12-24 14:55:00,NaT,DIRECT EMER.,P99698,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,...,511020.0,NaN,NaN,NaT,NaN,NaT,NaN,2011 - 2013,146,2011
4,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaT,EW EMER.,P35SU0,PACU,HOME,Other,...,258720.0,NaN,NaN,NaT,NaN,NaT,NaN,2011 - 2013,99,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73176,19999442,26785317,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,ELECTIVE,P147HV,PHYSICIAN REFERRAL,REHAB,Medicaid,...,1319100.0,NaT,NaN,NaT,NaN,NaT,NaN,2008 - 2010,138,2010
73177,19999625,25304202,2139-10-10 18:06:00,2139-10-16 03:30:00,NaT,EW EMER.,P46LSC,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,...,465840.0,NaT,NaN,16:44:00,-4920.0,19:18:00,4320.0,2008 - 2010,130,2009
73178,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,...,864960.0,NaT,NaN,09:11:00,-27180.0,18:12:00,5280.0,2017 - 2019,130,2019
73179,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,...,604500.0,13:42:00,604500.0,11:09:00,-9480.0,14:46:00,3540.0,2008 - 2010,156,2008


In [18]:
adm_overlap = pd.merge(iii_adm[adm_sub_ids], carevue_adm, on=adm_sub_ids)
just_iii_adm = iii_adm[~iii_adm.hadm_id.isin(adm_overlap.hadm_id)]

pre2012_icu_admissions = adm_icu[adm_icu['earliest_possible_dischyear'] <= 2012]

earliest_possible_estimate = unique_patient_adms(pre2012_icu_admissions).rename('earliest_possible')
carevue_estimate = unique_patient_adms(just_iii_adm).rename('carevue estimate')

pd.concat((
    pd.Series(
        (
            counts_with_percentage(icu_patients.anchor_year_group).loc[contaminated_ranges].sum()['total'],
            counts_with_percentage(adm_icu.anchor_year_group).loc[contaminated_ranges].sum()['total']
        ),
        index=['subject_id', 'hadm_id'], dtype=int, name='anchor_year_group'),

    earliest_possible_estimate,
    carevue_estimate,

    round(earliest_possible_estimate / unique_patient_adms(icu_stays) * 100, 1).rename('epe/ICU'),
    round(carevue_estimate / unique_patient_adms(icu_stays) * 100, 1).rename('cve/ICU')

), axis=1)

,anchor_year_group,earliest_possible,carevue estimate,epe/ICU,cve/ICU
subject_id,28372,22941,23294,45.1,45.7
hadm_id,43925,28877,32140,43.6,48.5


In [19]:
carevue_patients.nunique()

row_id         23692
subject_id     23692
gender             2
dob            19754
dod             7946
dod_hosp        4680
dod_ssn         7449
expire_flag        2
dtype: int64

In [20]:
post2012_icu_admissions = adm_icu[adm_icu['earliest_possible_dischyear'] > 2012]
unique_patient_adms(post2012_icu_admissions)

subject_id    29986
hadm_id       37362
Name: total, dtype: int64

# ICD Spread

In [21]:
pre2012_diagnoses = pd.merge(pre2012_icu_admissions[adm_sub_ids], admissions, on=adm_sub_ids)
post2012_diagnoses = pd.merge(post2012_icu_admissions[adm_sub_ids], admissions, on=adm_sub_ids)

In [22]:
iii_totals = pd.concat(
    (
        unique_patient_adms(iii_adm).rename('total_iii'),
        unique_patient_adms(just_iii_adm).rename('anti_carevue'),
        unique_patient_adms(carevue_adm).rename('carevue')
    ), axis=1
)
iii_totals['overlap'] = iii_totals.total_iii - (iii_totals.carevue + iii_totals.anti_carevue)
iii_totals['total_carevue'] = (iii_totals.carevue + iii_totals.anti_carevue)
iii_totals

,total_iii,anti_carevue,carevue,overlap,total_carevue
subject_id,46520,23294,23692,-466,46986
hadm_id,58976,32140,26836,0,58976


In [23]:
iii_totals_wo_newborn = pd.concat(
    (
        unique_patient_adms(iii_adm[iii_adm.admission_type != 'NEWBORN']).rename('total_iii'),
        unique_patient_adms(just_iii_adm[just_iii_adm.admission_type != 'NEWBORN']).rename('anti_carevue'),
        unique_patient_adms(carevue_adm[carevue_adm.admission_type != 'NEWBORN']).rename('carevue')
    ), axis=1
)
iii_totals_wo_newborn['overlap'] = iii_totals_wo_newborn.total_iii - (
        iii_totals_wo_newborn.carevue + iii_totals_wo_newborn.anti_carevue)
iii_totals_wo_newborn['total_carevue'] = (iii_totals_wo_newborn.carevue + iii_totals_wo_newborn.anti_carevue)
iii_totals_wo_newborn

,total_iii,anti_carevue,carevue,overlap,total_carevue
subject_id,38888,23202,16040,-354,39242
hadm_id,51113,32047,19066,0,51113


In [24]:
iii_adm.admission_type.value_counts()

admission_type
EMERGENCY    42071
NEWBORN       7863
ELECTIVE      7706
URGENT        1336
Name: count, dtype: int64

In [25]:
round((iii_totals.anti_carevue + iii_totals.overlap) / iii_totals.total_iii * 100, 1)

subject_id    49.1
hadm_id       54.5
dtype: float64

In [31]:
admissions_9 = admissions[(admissions.dia_version != 10) & (admissions.pro_version != 10)]

In [35]:
match_columns = ['admitmoment', 'dischmoment', 'dischinterval',]                 
# match_columns = moments + intervals                 

matching_admissions = pd.merge(admissions_9, iii_adm, on=match_columns, suffixes=('', '_iii'))
matching_admissions = matching_admissions[sorted(matching_admissions.columns)]
unique_patient_adms(matching_admissions)

subject_id    124703
hadm_id       277164
Name: total, dtype: int64

In [36]:
matching_admissions_9ap = matching_admissions[((matching_admissions.pro_codes == matching_admissions.pro_codes_iii) & (matching_admissions.dia_codes == matching_admissions.dia_codes_iii))]
len(matching_admissions_9ap)

23918

In [27]:
filter_columns = ['diagnosis', 'admit_provider_id', 'row_id', 'race', 'religion', 'admission_type_iii',
                  'admission_type_iv', 'has_chartevents_data', 'icd_version', 'icd9_code', 'icd_code',
                  'year_diff'] + match_columns
cols = sorted([col for col in matching_admissions.columns if
               col not in filter_columns and 'time' not in col and 'moment' not in col and 'subject_id' not in col])
matching_admissions[cols]

,admission_location,admission_location_iii,admission_type,anchor_year_group,chartdate,deathinterval,deathinterval_iii,dia_codes,dia_codes_iii,dia_version,...,language_iii,marital_status,marital_status_iii,pro_codes,pro_codes_iii,pro_version,seq_num_x,seq_num_y,seq_numdia,seq_numpro
0,WALK-IN/SELF REFERRAL,CLINIC REFERRAL/PREMATURE,EW EMER.,2017 - 2019,NaN,NaN,NaN,"{Z8546, G3183, R296, F0280, E785, R441}","{11284, 27401, 78009, 5854, 29421, 2760, 27739...",10.0,...,ENGL,MARRIED,SINGLE,{},{4516},NaN,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",{1},"{1, 2, 3, 4, 5, 6}",NaN
1,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,2008 - 2010,{2195-10-31},NaN,NaN,"{11284, 27401, 78009, 5854, 29421, 2760, 27739...","{11284, 27401, 78009, 5854, 29421, 2760, 27739...",9.0,...,ENGL,SINGLE,SINGLE,{4516},{4516},9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",{1},"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{1}
2,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,SURGICAL SAME DAY ADMISSION,2011 - 2013,{2190-07-19},NaN,NaN,"{73399, 73819, V153, 5559, V8741, 99832, 1123,...","{44101, 60001, 5939, 27800, 25000, 4019, 7213,...",9.0,...,ENGL,MARRIED,MARRIED,"{0207, 8674}","{9604, 5794, 8872}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0}","{1, 2, 3}","{1, 2, 3, 4, 5, 6, 7, 8}","{1, 2}"
3,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2008 - 2010,{2167-03-17},NaN,NaN,"{5723, 5715, 5856, 28521, V4511, 56984, 25208,...","{44101, 60001, 5939, 27800, 25000, 4019, 7213,...",9.0,...,ENGL,SINGLE,MARRIED,{0689},"{9604, 5794, 8872}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0}","{1, 2, 3}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}",{1}
4,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,SURGICAL SAME DAY ADMISSION,2011 - 2013,{2151-03-29},NaN,NaN,{1890},"{44101, 60001, 5939, 27800, 25000, 4019, 7213,...",9.0,...,ENGL,SINGLE,MARRIED,"{5551, 403}","{9604, 5794, 8872}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0}","{1, 2, 3}",{1},"{1, 2}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50796,EMERGENCY ROOM,EMERGENCY ROOM ADMIT,EW EMER.,2008 - 2010,{2132-03-31},NaN,NaN,"{2859, 5712, 5780, 5771, 25000, 2875}","{2859, 5712, 5780, 5771, 25000, 2875}",9.0,...,ENGL,SINGLE,SINGLE,{4513},{4513},9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0}",{1},"{1, 2, 3, 4, 5, 6}",{1}
50797,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,2011 - 2013,{2161-08-25},NaN,NaN,"{920, 53081, 2910, 37230, 7813, 79319, 30391, ...","{920, 53081, 2910, 37230, 7813, 79319, 30391, ...",9.0,...,ENGL,DIVORCED,DIVORCED,"{9671, 9604}","{9671, 9604}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}","{1, 2}"
50798,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,ELECTIVE,2008 - 2010,"{2148-11-22, 2148-11-19}",NaN,NaN,"{11284, 43491, 5990, 34881, 7843, 311, 34541, ...","{11284, 43491, 5990, 34881, 7843, 311, 34541, ...",9.0,...,ENGL,DIVORCED,SINGLE,"{9671, 0153, 966, 9604}","{9671, 0153, 966, 9604}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2, 3, 4}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13}","{1, 2, 3, 4}"
50799,EMERGENCY ROOM,EMERGENCY ROOM ADMIT,EW EMER.,2008 - 2010,"{2139-10-14, 2139-10-10}",NaN,NaN,"{486, 5849, 70720, V4589, 32742, 33182, 2760, ...","{486, 5849, 70720, V4589, 32742, 33182, 2760, ...",9.0,...,GREE,MARRIED,MARRIED,"{966, 9390}","{966, 9390}",9.0,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1, 2}"


In [41]:
matching_admissions_9ap.earliest_possible_dischyear.value_counts()

earliest_possible_dischyear
2008    11289
2011     6350
2009     3018
2010     1866
2012     1228
2013      130
2014       13
2007        9
2005        6
2006        4
2016        3
2017        2
Name: count, dtype: int64

In [33]:
unique_patient_adms(matching_admissions_9ap)

subject_id    18502
hadm_id       23413
Name: total, dtype: int64

In [34]:
icu_matching = pd.merge(icu_stays[adm_sub_ids + ['los']], matching_admissions_9ap, on=adm_sub_ids, how='outer')

In [35]:
unique_patient_adms(pre2012_icu_admissions)

subject_id    22941
hadm_id       28877
Name: total, dtype: int64

In [36]:
pre2012_icu_admissions.earliest_possible_dischyear.value_counts()

earliest_possible_dischyear
2011    12349
2008    10166
2012     3603
2009     3304
2010     2296
Name: count, dtype: int64

In [37]:
pre2012_matching = pd.merge(pre2012_icu_admissions[adm_sub_ids + ['earliest_possible_dischyear']], matching_admissions_9ap, on=adm_sub_ids, how='outer', suffixes=['_pre', ''])

In [38]:
unique_patient_adms(pre2012_matching)

subject_id    24977
hadm_id       31936
Name: total, dtype: int64

In [39]:
matching_admissions_9ap[matching_admissions_9ap.earliest_possible_dischyear == 2013]

,admission_location,admission_location_iii,admission_type,admission_type_iii,admit_provider_id,admitmoment,admittime,admittime_iii,anchor_year_group,chartdate,...,pro_version,race,religion,seq_num_x,seq_num_y,seq_numdia,seq_numpro,subject_id,subject_id_iii,year_diff
2202,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,ELECTIVE,ELECTIVE,P34LOL,00:00:00,2112-04-09 00:00:00,2183-10-14 00:00:00,2011 - 2013,{2112-04-09},...,9.0,OTHER,CATHOLIC,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0}",{1},"{1, 2, 3, 4, 5, 6, 7}",{1},16361542,63637,99
2357,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,SURGICAL SAME DAY ADMISSION,ELECTIVE,P47SIK,07:15:00,2134-03-23 07:15:00,2119-07-24 07:15:00,2008 - 2010,{2134-03-23},...,9.0,BLACK/AFRICAN AMERICAN,NOT SPECIFIED,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2, 3}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}","{1, 2, 3}",19209496,70807,121
4291,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,ELECTIVE,ELECTIVE,P34LOL,11:00:00,2112-03-19 11:00:00,2183-09-23 11:00:00,2011 - 2013,{2112-03-19},...,9.0,OTHER,CATHOLIC,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0}","{1, 2}","{1, 2, 3, 4, 5, 6, 7, 8, 9}","{1, 2}",16361542,63637,99
9375,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,EMERGENCY,P34SFE,18:41:00,2156-01-07 18:41:00,2150-07-03 18:41:00,2011 - 2013,"{2156-01-08, 2156-01-19}",...,9.0,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1, 2}",10310992,55523,143
10111,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,EMERGENCY,P03G4D,03:34:00,2172-01-29 03:34:00,2138-03-16 03:34:00,2008 - 2010,{2172-02-01},...,9.0,WHITE,NOT SPECIFIED,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1, 2}",10388429,5193,159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49313,PHYSICIAN REFERRAL,PHYS REFERRAL/NORMAL DELI,SURGICAL SAME DAY ADMISSION,ELECTIVE,P60ZCO,10:00:00,2154-02-25 10:00:00,2179-06-21 10:00:00,2008 - 2010,{2154-02-25},...,9.0,WHITE,CATHOLIC,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2, 3}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1, 2, 3}",19391968,76479,141
49316,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,EMERGENCY,P3529J,17:38:00,2154-01-08 17:38:00,2179-05-04 17:38:00,2008 - 2010,{2154-01-08},...,9.0,WHITE,CATHOLIC,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",{1},"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",{1},19391968,76479,141
50230,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,EMERGENCY,P27588,04:07:00,2161-02-08 04:07:00,2173-11-02 04:07:00,2011 - 2013,NaN,...,NaN,WHITE,JEWISH,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",NaN,"{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",NaN,19774387,44741,148
50343,EMERGENCY ROOM,CLINIC REFERRAL/PREMATURE,EW EMER.,EMERGENCY,P072C5,00:19:00,2185-01-03 00:19:00,2195-07-15 00:19:00,2008 - 2010,"{2185-01-03, 2185-01-06, 2185-01-04}",...,9.0,BLACK/AFRICAN AMERICAN,PROTESTANT QUAKER,"{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","{1, 2, 3, 4, 5}","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{1, 2, 3, 4, 5}",19819996,20124,172


In [40]:
pre2012_matching[(pre2012_matching.earliest_possible_dischyear.isna())].earliest_possible_dischyear_pre.value_counts()

earliest_possible_dischyear_pre
2011.0    5544
2012.0    2258
2008.0     799
2010.0     330
2009.0     312
Name: count, dtype: int64

In [41]:
pre2012_matching[pre2012_matching.earliest_possible_dischyear.isna()]

,subject_id,hadm_id,earliest_possible_dischyear_pre,admission_location,admission_location_iii,admission_type,admission_type_iii,admit_provider_id,admitmoment,admittime,...,pro_codes_iii,pro_version,race,religion,seq_num_x,seq_num_y,seq_numdia,seq_numpro,subject_id_iii,year_diff
0,10000980,26913865,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001217,24597018,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001217,27703517,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001725,25563031,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10002155,20345487,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31700,19998330,21135114,2012.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31708,19998770,28494258,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31709,19998843,24842066,2011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31713,19999297,21439025,2008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
idx = 0
print(notes.loc[20345487].text)

In [ ]:
print('\n\n'.join(iii_notes[iii_notes.hadm_id == 20345487
].text))
# iii_notes[iii_notes.text.str.contains('\?CAD, peripheral vascular')]

In [ ]:
unique_patient_adms(pd.merge(icu_matching, pre2012_matching, on=adm_sub_ids)[(pd.merge(icu_matching, pre2012_matching, on=adm_sub_ids).los.isna() & pd.merge(icu_matching, pre2012_matching, on=adm_sub_ids).admittime_pre.isna())])

In [ ]:
pre2012_matching.admittime_pre.isna().sum()

In [ ]:
idx = 10
print(matching_admissions_9ap.loc[idx].dia_codes)
print(matching_admissions_9ap.loc[idx].dia_codes_iii)
print(matching_admissions_9ap.loc[idx].dia_codes == matching_admissions_9.loc[idx].dia_codes_iii)

In [ ]:
print('\n\n'.join(iii_notes[iii_notes.hadm_id == matching_admissions_9ap.loc[idx].hadm_id_iii].text))

In [ ]:
print(notes.loc[matching_admissions_9ap.loc[idx].hadm_id].text)

In [ ]:
counts_with_percentage(matching_admissions.earliest_possible_dischyear)

In [ ]:
unique_patient_adms(matching_admissions, ['subject_id_iii', 'hadm_id_iii'])

In [ ]:
unique_patient_adms(matching_admissions, ['subject_id', 'hadm_id'])

In [ ]:
matching_and_pre2012 = pd.merge(matching_admissions, pre2012_icu_admissions, how='outer', on=adm_sub_ids)
unique_patient_adms(matching_and_pre2012, ['subject_id', 'hadm_id'])

# Notes

In [39]:
mimic4_notes_dir = data_dir / 'mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/'

notes = pd.read_csv(mimic4_notes_dir / 'discharge.csv.gz', index_col='hadm_id')

In [40]:
iii_notes = pd.read_csv(mimiciii_dir / 'NOTEEVENTS.csv')
iii_notes.columns = iii_notes.columns.str.lower()
iii_disch_summaries = iii_notes[iii_notes.category == 'Discharge summary']

C:\Users\tsteffek\AppData\Local\Temp\ipykernel_49448\1037434142.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  iii_notes = pd.read_csv(mimiciii_dir / 'NOTEEVENTS.csv')


In [ ]:
print(notes.loc[28494258].text)
print('\n\n'.join(iii_notes[iii_notes.hadm_id == 150767.0].text))

In [ ]:
iii_notes[iii_notes.text.str.contains('Temporal Lobe Epilepsy')]

In [ ]:
print('\n\n'.join(iii_notes[iii_notes.hadm_id == 150767.0].text))

In [ ]:
icu_matching_diagnoses = pd.merge(matching_admissions, icu_stays[adm_sub_ids + ['los']], how='left',
                                  right_on=adm_sub_ids, left_on=[i + '_iv' for i in adm_sub_ids])
icu_matching_diagnoses[icu_matching_diagnoses.los.isna()][['hadm_id_iii', 'hadm_id_iv']]

In [ ]:
icu_matching_diagnoses

In [ ]:
counts_with_percentage(matching_admissions.anchor_year_group)

In [ ]:
matching_admissions.sort_values('str_icd_codes', key=lambda x: x.str.len())[
    ['str_icd_codes', 'hadm_id_iii', 'hadm_id_iv']]

In [ ]:
print('\n\n'.join(iii_notes[iii_notes.hadm_id == 188170].text))

In [ ]:
print(notes.loc[25232863]['text'])

In [ ]:
iii_notes_with_diagnoses = pd.merge(iii_notes, iii_diagnoses, on=adm_sub_ids)

In [ ]:
epy_wo_matching = pre2012_icu_admissions[~pre2012_icu_admissions.hadm_id.isin(matching_admissions_9ap.hadm_id)]
epy_wo_matching

In [ ]:
idx += 1
iv_id = epy_wo_matching.iloc[idx].hadm_id
idx

In [ ]:
print(notes.loc[iv_id].text)

In [ ]:
text = 'cell lung cancer, CAD and CKD who presents with'
detected_summaries = iii_disch_summaries[iii_disch_summaries.text.str.contains(text.replace('(', '\(').replace(')', '\)').replace(' ', '\s*'))]
iii_idx = 0; iii_id = detected_summaries.hadm_id.iloc[iii_idx]
print(detected_summaries.hadm_id, '\n\n'); print(detected_summaries.iloc[iii_idx].text)

In [ ]:
iv = admissions[admissions.hadm_id == iv_id].iloc[0]
iii = iii_adm[iii_adm.hadm_id == iii_id].iloc[0]
print('dia_codes', iii.dia_codes == iv.dia_codes,'\n', iii.dia_codes, '\n',iv.dia_codes)
print('pro_codes', iii.pro_codes == iv.pro_codes, '\n',iii.pro_codes, '\n',iv.pro_codes)
for moment in moments:
    print(moment, iii[moment] == iv[moment], iii[moment], iv[moment])
for interval in intervals:
    print(interval, iii[interval] == iv[interval], iii[interval], iv[interval])

In [ ]:
print(iii_adm[iii_adm.hadm_id == 196292].text.iloc[0])

In [ ]:
iii_adm_but_not_matching = just_iii_adm[~just_iii_adm.hadm_id.isin(matching_admissions_9ap.hadm_id_iii)]
iii_adm_but_not_matching.iloc[80:91]

In [ ]:
iii_id = 111426
print(iii_disch_summaries[iii_disch_summaries.hadm_id == iii_id].text.iloc[0])

In [ ]:
text = 'She was was started on antifungals receiving her first'
notes[notes.text.str.contains(text.replace('(', '\(').replace(')', '\)').replace(' ', '\s*'))]

In [ ]:
iv_id = 28447722
print(notes.loc[iv_id].text)

In [ ]:
iv = admissions[admissions.hadm_id == iv_id].iloc[0]
iii = iii_adm[iii_adm.hadm_id == iii_id].iloc[0]
print('dia_codes', iii.dia_codes == iv.dia_codes,'\n', iii.dia_codes, '\n',iv.dia_codes)
print('pro_codes', iii.pro_codes == iv.pro_codes, '\n',iii.pro_codes, '\n',iv.pro_codes)
for interval in intervals:
    print(interval, iii[interval] == iv[interval], iii[interval], iv[interval])

In [ ]:
iv

In [ ]:
iii

In [ ]:
pd.merge(adm_icu[adm_icu.hadm_id == iv_id], iii_adm[iii_adm.hadm_id == iii_id], on=intervals)